# Sistemas Inteligentes

## Practica 1: Búsqueda en espacio de estados

# Estructura

## Clases



### Acción
Clase Acción que representa que es lo que ha de ser hecho, en este caso el origen, el destino y el coste entre estos.




### Estado
Clase estado que contiene el estado del problema, que en este caso es la interseccion donde estamos + el id


### Nodo
Clase que representa un nodo dentro del grafo a resolver. Este contiene un estado, un padre, una accion y su profundidad en el grafo.


### Problema
Clase que representa el problema a resolver, contiene la informacion del JSON y 


### Busqueda
Clase abstracta que contiene el bucle basico que usan todos los metodos de busqueda,

### 1: Estrategia de búsqueda no informada

#### 1.1: Búsqueda primero en anchura

In [14]:
from Classes import *
from Busqueda import Busqueda_Anchura
#Windows
ruta_json = r'problems\medium\calle_mariÌa_mariÌn_500_0.json'
#MacOs
#ruta_json = r'pr1_SSII/problems/medium/calle_mariÌa_mariÌn_500_0.json'

# Crear el objeto problema
problema = Problema(ruta_json)

anchura = Busqueda_Anchura(problema)

# Ejecutar el algoritmo de búsqueda
solucion, soluciones_generadas, nodos_explorados, nodos_expandidos, coste, profundidad = anchura.buscar()

# Mostrar la solución y estadísticas
if solucion:
    print("Solución encontrada:", solucion)
    print("Estadísticas:")
    print(f"Número de soluciones generadas: {soluciones_generadas}")
    print(f"Número de nodos explorados: {nodos_explorados}")
    print(f"Número de nodos expandidos: {nodos_expandidos}")
    print(f"Coste de la solución: {coste}")
    print(f"Profundidad de la solución: {profundidad}")
else:
    print("No se encontró solución.")


 añadido a cerrados

 sigo, profundidad 0

 añadido a cerrados

 sigo, profundidad 1

 añadido a cerrados

 sigo, profundidad 1

 añadido a cerrados

 sigo, profundidad 2

 añadido a cerrados

 sigo, profundidad 2

 añadido a cerrados

 sigo, profundidad 3

 añadido a cerrados

 sigo, profundidad 3

 añadido a cerrados

 sigo, profundidad 3

 añadido a cerrados

 sigo, profundidad 4

 añadido a cerrados

 sigo, profundidad 4

 añadido a cerrados

 sigo, profundidad 4

 añadido a cerrados

 sigo, profundidad 4

 añadido a cerrados

 sigo, profundidad 4

 añadido a cerrados

 sigo, profundidad 5

 añadido a cerrados

 sigo, profundidad 5

 añadido a cerrados

 sigo, profundidad 5

 sigo, profundidad 5

 añadido a cerrados

 sigo, profundidad 5

 añadido a cerrados

 sigo, profundidad 5

 añadido a cerrados

 sigo, profundidad 5

 sigo, profundidad 5

 sigo, profundidad 5

 añadido a cerrados

 sigo, profundidad 5

 añadido a cerrados

 sigo, profundidad 6

 añadido a cerrados

 sigo, pr